In [3]:
import pandas as pd
import functions

/Users/maggiemyslik/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# LOAD VDEM:

In [2]:
vdem = pd.read_csv("../data/raw_downloads/vdem.csv")

vdem = vdem[[
    'country_name', 
    'country_text_id', 
    'country_id', 
    'year', 
    'v2x_polyarchy',      # Electoral Democracy Index
    'v2x_libdem',         # Liberal Democracy Index
    'v2x_partipdem',      # Participatory Democracy Index
    'v2x_delibdem',       # Deliberative Democracy Index
    'v2x_egaldem',        # Egalitarian Democracy Index
    'v2x_regime',         # Regime Type
    'v2xel_frefair',      # Free and Fair Elections
    'v2x_clpriv',         # Civil Liberties
    'v2x_accountability', # Accountability
    'v2x_civlib'          # Civil Liberties Index
]]

vdem.rename(columns={
    'v2x_polyarchy': 'electoral_democracy',
    'v2x_libdem': 'liberal_democracy',
    'v2x_partipdem': 'participatory_democracy',
    'v2x_delibdem': 'deliberative_democracy',
    'v2x_egaldem': 'egalitarian_democracy',
    'v2x_regime': 'regime_type',
    'v2xel_frefair': 'free_and_fair_elections',
    'v2x_clpriv': 'civil_liberties',
    'v2x_accountability': 'accountability',
    'v2x_civlib': 'civil_liberties_index'
}, inplace=True)

vdem.to_csv("../data/vdem_small.csv", index=False)

# LOAD OTHER DEMOCRACY MEASURES:
## 1. Global State of Democracy (GSOD) Indices: Democracy Tracker 

- https://www.idea.int/democracytracker/gsod-indices/ 
- https://www.idea.int/data-tools/tools/global-state-democracy-indices 
- [code book](https://www.idea.int/sites/default/files/2024-09/gsod-indices-codebook-v8-2024.pdf)

In [3]:
gsod = pd.read_excel("../data/raw_downloads/gsod.xlsx")

selected_columns = {
    'country_name': 'country',  
    'year': 'year',  
    'representation_est': 'representation_index',  # Overall representation index
    'cred_elect_est': 'credible_elections',  # Credibility of elections
    'inclu_suff_est': 'inclusive_suffrage',  # Inclusiveness of suffrage
    'free_parties_est': 'free_political_parties',  # Freedom of political parties
    'rights_est': 'rights_index',  # Overall rights index
    'civil_lib_est': 'civil_liberties',  # Civil liberties
    'free_express_est': 'freedom_of_expression',  # Freedom of expression
    'rule_law_est': 'rule_of_law',  # Rule of law
    'jud_ind_est': 'judicial_independence',  # Judicial independence
    'abs_corrupt_est': 'corruption_level',  # Levels of corruption
    'participation_est': 'participation_index',  # Overall participation index
    'civil_soc_est': 'civil_society_engagement',  # Civil society engagement
    'basic_welf_est': 'basic_welfare',  # Provision of basic welfare
    'pol_equal_est': 'political_equality'  # Political equality
}

gsod = gsod[list(selected_columns.keys())].rename(columns=selected_columns)

## 2. EIU Report: Democracy Index
- https://www.eiu.com/n/campaigns/democracy-index-2023/

In [4]:
eiu = pd.read_excel("../data/raw_downloads/eiu.xlsx")
year_columns = [col for col in eiu.columns if col.isdigit()]
eiu = pd.melt(
    eiu,
    id_vars=['Economy ISO3', 'Economy Name'],  # Columns to keep
    value_vars=year_columns,  # Columns to unpivot
    var_name='year',          # Name for the year column
    value_name='eiu_rating'       # Name for the rating column
)
eiu.rename(
    columns={
        'Economy ISO3': 'country_code',
        'Economy Name': 'country',
        'Indicator ID': 'indicator_id',
        'Indicator': 'indicator'
    },
    inplace=True
)

eiu['year'] = eiu['year'].astype(int)

## 3. Vanahan Measures
- [codebook](https://xmarquez.github.io/democracyData/reference/vanhanen.html)
- [dataset](https://services.fsd.tuni.fi/catalogue/FSD1289?tab=summary&lang=en&study_language=en)

In [38]:
vanahan = pd.read_csv("../data/raw_downloads/vanahan.csv", delimiter=";")

democracy_columns = [col for col in vanahan.columns if col.startswith("q") and col.endswith("_3")]
relevant_columns = ["bv1"] + democracy_columns
vanahan = vanahan[relevant_columns]

vanahan = vanahan.melt(
    id_vars=["bv1"], 
    value_vars=democracy_columns, 
    var_name="year_column", 
    value_name="democracy_index"
)

vanahan["year"] = vanahan["year_column"].str.extract(r"q(\d+)_3").astype(int) + 1809
vanahan = vanahan.drop(columns=["year_column"])
vanahan = vanahan.rename(columns={"bv1": "country"})

vanahan['democracy_index'] = vanahan['democracy_index'].str.replace(",", "", regex=False)
vanahan['democracy_index'] = pd.to_numeric(vanahan['democracy_index'], errors='coerce')
vanahan = vanahan.dropna(subset=['democracy_index'])
vanahan = vanahan.reset_index(drop=True)
vanahan = vanahan[(vanahan['democracy_index'] >= 0) & (vanahan['democracy_index'] <= 100)]

## 4. Polity Score: 
- https://www.systemicpeace.org/inscrdata.html 
- https://ourworldindata.org/grapher/democracy-index-polity

In [5]:
polity = pd.read_csv("../data/raw_downloads/polity.csv")
polity.rename(
    columns={
        'Entity': 'country',
        'Code': 'country_code',
        'Year': 'year',
        'Democracy': 'polity_democracy_score'
    },
    inplace=True
)

## 5. Freedom House: Freedom in the World Report
https://freedomhouse.org/report/freedom-world#Data 

INDEX for 2013-2021:                                        
- F=Free, PF=Partly Free, NF=Not Free										
- PR Rating=Political Rights Rating										
- CL Rating=Civil Liberties Rating										
- A =aggregate score for the A. Electoral Process subcategory										
- B =aggregate score for the B. Political Pluralism and Participation subcategory										
- C =aggregate score for the C. Functioning of Government subcategory										
- Add Q (Add B)=score for Additional Discretionary Question (B)										
- Add A=score for Additional Discretionary Question A										
- PR =aggregate score for the Political Rights category										
- D =aggregate score for the D. Freedom of Expression and Belief subcategory										
- E =aggregate score for the E. Associational and Organizational Rights subcategory										
- F =aggregate score for the F. Rule of Law subcategory										
- G =aggregate score for te G. Personal Autonomy and Individual Rights subcategory										
- CL =aggregate score for the Civil Liberties category										
- Total =aggregate score for all categories	

In [6]:
freedom_house = pd.read_csv("../data/raw_downloads/fh.csv")
freedom_house.columns = freedom_house.iloc[0] 
freedom_house = freedom_house[1:].reset_index(drop=True)
columns = [
    "Edition",       # Year
    "PR",            # Political rights aggregate
    "CL",            # Civil liberties aggregate
    "Total",         # Overall score
    "A1",            # Electoral process
    "A2",            # Political pluralism and participation
    "A3",            # Functioning of government
    "D",             # Freedom of expression
    "E",             # Associational and organizational rights
    "F"              # Rule of law
]
freedom_house = freedom_house[columns]
freedom_house.rename(
    columns={
        "Edition": "year",
        "PR": "political_rights_aggregate",
        "CL": "civil_liberties_aggregate",
        "Total": "overall_score",
        "A1": "electoral_process_score",
        "A2": "political_pluralism_score",
        "A3": "functioning_of_government_score",
        "D": "freedom_of_expression_score",
        "E": "associational_rights_score",
        "F": "rule_of_law_score"
    },
    inplace=True
)

## 6. ACLP

https://sites.google.com/site/joseantoniocheibub/datasets/aclp?authuser=0 

CODEBOOK:
- YEAR -> year (Year of observation.)
- REG -> regime_type (0 for democracy, 1 for dictatorship.)
- INST -> institution_type (Classification of political regimes.)
- DIVIDED -> division_of_powers (Extent of formal power divisions in dictatorships.)
- AUT -> autocracy_type (Dictatorship classification based on legislature existence.)
- CIVLIB -> civil_liberties (Freedom House civil liberty scale, 1 to 7.)
- POLLIB -> political_liberties (Freedom House political liberty scale, 1 to 7.)
- EFFPARTY -> effective_parties (Number of effective political parties.)
- MOBILIZE -> party_mobilization (Distinguishes dictatorship types by party presence.)
- SEATS -> largest_party_seats (Percentage of lower house seats held by the largest party.)
- STRD -> transitions_to_democracy (Sum of past transitions to democracy.)
- STRA -> transitions_to_authoritarianism (Sum of past transitions to authoritarianism.)
- UNSTABLE -> political_instability (Indicates regime transitions from 1950 to 1990.)
- WAR -> war_present (Dummy variable for war presence in the country.)

In [7]:
aclp = pd.read_excel("../data/raw_downloads/aclp.xls")
columns = {
    "COUNTRY": "country",
    "YEAR": "year",
    "REG": "regime_type",
    "INST": "institution_type",
    "DIVIDED": "division_of_powers",
    "AUT": "autocracy_type",
    "CIVLIB": "civil_liberties",
    "POLLIB": "political_liberties",
    "EFFPARTY": "effective_parties",
    "MOBILIZE": "party_mobilization",
    "SEATS": "largest_party_seats",
    "STRD": "transitions_to_democracy",
    "STRA": "transitions_to_authoritarianism",
    "UNSTABLE": "political_instability",
    "WAR": "war_present",
}
aclp = aclp[columns.keys()].rename(columns=columns)
aclp['country'] = aclp['country'].map(functions.country_mapping)

# COMBINE DATA SETS